import pandas as pd

In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Cook Shop.csv', error_bad_lines=False);

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0      [happy, customer, excellent, quality, food, fr...
1      [food, delicious, unique, combinations, differ...
2      [food, soooo, delicious, waiter, recommended, ...
3      [business, lunch, service, good, staff, helpfu...
4      [great, experience, food, delicious, creative,...
                             ...                        
205    [great, dinner, excellent, menu, options, cove...
206    [excellent, homemade, bread, pork, chop, salmo...
207    [visited, place, times, planning, warm, friend...
208    [visited, cookshop, friends, lived, nicosia, l...
209    [nice, restaurant, good, food, wasn, huge, mea...
Name: r_text, Length: 210, dtype: object

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=250,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [13]:
from pprint import pprint

In [14]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.023*"food" + 0.018*"great" + 0.015*"service" + 0.014*"good" + '
  '0.012*"lunch" + 0.012*"atmosphere" + 0.012*"restaurant" + 0.012*"excellent" '
  '+ 0.012*"nicosia" + 0.011*"best"'),
 (1,
  '0.028*"food" + 0.020*"great" + 0.016*"place" + 0.015*"nice" + '
  '0.014*"restaurant" + 0.013*"service" + 0.013*"good" + 0.012*"best" + '
  '0.010*"lunch" + 0.009*"staff"'),
 (2,
  '0.027*"place" + 0.023*"food" + 0.017*"good" + 0.016*"excellent" + '
  '0.016*"service" + 0.014*"lunch" + 0.011*"menu" + 0.010*"quality" + '
  '0.010*"great" + 0.010*"best"'),
 (3,
  '0.016*"restaurant" + 0.015*"menu" + 0.008*"food" + 0.008*"ordered" + '
  '0.006*"visit" + 0.006*"place" + 0.006*"burgers" + 0.006*"summary" + '
  '0.006*"recently" + 0.006*"better"'),
 (4,
  '0.022*"place" + 0.018*"food" + 0.011*"service" + 0.010*"tasty" + '
  '0.009*"dishes" + 0.008*"good" + 0.008*"plates" + 0.008*"best" + '
  '0.007*"great" + 0.007*"cooked"'),
 (5,
  '0.020*"place" + 0.020*"food" + 0.017*"good" + 0.015*"pork" +

In [15]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.037107 -0.101246       1        1  21.606321
1      0.048191  0.004242       2        1  21.176142
2      0.027767  0.073443       3        1  19.516846
5      0.044804  0.011205       4        1  16.852434
0      0.032696  0.005165       5        1  12.420396
4     -0.074500  0.040345       6        1   5.078176
3     -0.116065 -0.033154       7        1   3.349686, topic_info=           Term       Freq      Total Category  logprob  loglift
94         menu  41.000000  41.000000  Default  30.0000  30.0000
22        place  91.000000  91.000000  Default  29.0000  29.0000
26   restaurant  58.000000  58.000000  Default  28.0000  28.0000
322     amazing  28.000000  28.000000  Default  27.0000  27.0000
23         pork  24.000000  24.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
287        best   0.603965  41.378708   Topic7  -5.6490  -0.8307
108     stuffed   0.547106  11.474294   Topic7  -5.7479   0.3531
101      people   0.546112  11.155937   Topic7  -5.7497   0.3794
73        salad   0.555503  26.749996   Topic7  -5.7327  -0.4781
23         pork   0.552443  24.387869   Topic7  -5.7382  -0.3912

[505 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1156      6  0.771059  achieves
381       5  0.380099  actually
381       6  0.380099  actually
580       3  0.732216     added
580       4  0.244072     added
...     ...       ...       ...
1140      4  0.628205    wouldn
1155      7  0.820214     wrong
221       2  0.280892     years
221       3  0.561785     years
221       7  0.140446     years

[804 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 3, 6, 1, 5, 4])